In [ ]:
import pandas as pd
import numpy as np

df = pd.read_csv("/content/amazon-web-traffic-dataset.csv")
df.head(10)

,Country,Timestamp,Device Category,Key Actions,Page Path,Source,Avg Session Duration,Bounce Rate,Conversions,New Users,Page Views,Returning Users,Unique Page Views,Average time on home page (min),Website,Date,Time,Day
0,India,01-01-2019 05:01,Mobile,Purchase,/home,direct,137,35,42,804,1984,307,243,4.41,amazon.com,01-01-2019,05:01:00,Tuesday
1,USA,01-01-2019 07:52,Tablet,Sign Up,/products,referral,179,53,15,438,1938,190,1194,4.89,Amazon Prime,01-01-2019,07:52:00,Tuesday
2,India,02-01-2019 06:55,Mobile,Sign Up,/contact,organic search,84,49,2,878,1078,191,450,4.35,Amazon Prime,02-01-2019,06:55:00,Wednesday
3,India,03-01-2019 18:09,Mobile,Subscribe,/contact,social media,168,35,15,779,372,135,479,5.66,Amazon Web Services (AWS),03-01-2019,18:09:00,Thursday
4,India,03-01-2019 23:36,Desktop,Contact Form,/contact,direct,118,42,40,570,1225,20,147,5.19,AWS Support,03-01-2019,23:36:00,Thursday
5,India,04-01-2019 08:46,Desktop,Sign Up,/about,direct,160,48,33,208,1678,167,709,5.54,Amazon Music,04-01-2019,08:46:00,Friday
6,UK,04-01-2019 09:46,Mobile,Purchase,/blog,organic search,67,20,4,635,1291,432,451,0.90,amazon.com,04-01-2019,09:46:00,Friday
7,Canada,04-01-2019 15:14,Mobile,Sign Up,/home,referral,76,56,19,673,1271,282,866,2.49,Amazon.com,04-01-2019,15:14:00,Friday
8,Australia,05-01-2019 02:55,Mobile,Download,/about,social media,110,42,20,232,1307,335,1066,4.58,Prime Video,05-01-2019,02:55:00,Saturday
9,India,05-01-2019 04:28,Mobile,Purchase,/contact,organic search,136,58,31,83,525,425,1453,4.29,amazon.com,05-01-2019,04:28:00,Saturday


In [ ]:
print(df.columns)

Index(['Country', 'Timestamp', 'Device Category', 'Key Actions', 'Page Path',
       'Source', 'Avg Session Duration', 'Bounce Rate', 'Conversions',
       'New Users', 'Page Views', 'Returning Users', 'Unique Page Views',
       'Average time on home page (min)', 'Website', 'Date', 'Time', 'Day'],
      dtype='object')


In [ ]:
df.dtypes

,0
Country,object
Timestamp,object
Device Category,object
Key Actions,object
Page Path,object
Source,object
Avg Session Duration,int64
Bounce Rate,int64
Conversions,int64
New Users,int64


In [ ]:
obj_cols = df.select_dtypes(include='object').columns
print(obj_cols)

df[obj_cols] = df[obj_cols].apply(lambda x: x.str.strip())
print(df.head(10))

Index(['Country', 'Timestamp', 'Device Category', 'Key Actions', 'Page Path',
       'Source', 'Website', 'Date', 'Time', 'Day'],
      dtype='object')
     Country         Timestamp Device Category   Key Actions  Page Path  \
0      India  01-01-2019 05:01          Mobile      Purchase      /home   
1        USA  01-01-2019 07:52          Tablet       Sign Up  /products   
2      India  02-01-2019 06:55          Mobile       Sign Up   /contact   
3      India  03-01-2019 18:09          Mobile     Subscribe   /contact   
4      India  03-01-2019 23:36         Desktop  Contact Form   /contact   
5      India  04-01-2019 08:46         Desktop       Sign Up     /about   
6         UK  04-01-2019 09:46          Mobile      Purchase      /blog   
7     Canada  04-01-2019 15:14          Mobile       Sign Up      /home   
8  Australia  05-01-2019 02:55          Mobile      Download     /about   
9      India  05-01-2019 04:28          Mobile      Purchase   /contact   

           Source  Avg

In [ ]:
initial_len = len(df)
print("initial length:", initial_len)

df = df.dropna()
df = df.drop_duplicates()

len(df)

initial length: 2006


2006

In [ ]:
cleaning_logs = []

# Convert date and time columns
try:
    df['Timestamp'] = pd.to_datetime(df['Timestamp'], errors='coerce')
    df['Date'] = pd.to_datetime(df['Date'], errors='coerce')
    cleaning_logs.append("Converted 'Timestamp' and 'Date' to datetime format.")
except Exception as e:
    cleaning_logs.append(f"Failed to convert dates: {e}")

# Strip leading/trailing whitespace from object columns
obj_cols = df.select_dtypes(include='object').columns
df[obj_cols] = df[obj_cols].apply(lambda x: x.str.strip())
cleaning_logs.append(f"Stripped whitespace from object columns: {list(obj_cols)}")

# standard text
categorical_columns = ['Country', 'Device Category', 'Key Actions', 'Source', 'Website', 'Day']
for col in categorical_columns:
    if col in df.columns:
        df[col] = df[col].str.lower()
        cleaning_logs.append(f"Standardized casing for '{col}'")

# remove missing values
missing_report = df.isnull().sum()
for col, missing in missing_report.items():
    if missing > 0:
        if df[col].dtype in ['float64', 'int64']:
            df[col] = df[col].fillna(df[col].median())
            cleaning_logs.append(f"Filled {missing} missing values in '{col}' with median.")
        else:
            df[col] = df[col].fillna("unknown")
            cleaning_logs.append(f"Filled {missing} missing values in '{col}' with 'unknown'.")

# remove invalide dates
before = len(df)
df = df[df['Timestamp'].notnull()]
df = df[df['Date'].notnull()]
cleaning_logs.append(f"Removed {before - len(df)} rows with invalid dates.")

# capping
numeric_cols = df.select_dtypes(include=['int64', 'float64']).columns
for col in numeric_cols:
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    lower = Q1 - 1.5 * IQR
    upper = Q3 + 1.5 * IQR
    outliers = ((df[col] < lower) | (df[col] > upper)).sum()
    df[col] = np.where(df[col] < lower, lower, df[col])
    df[col] = np.where(df[col] > upper, upper, df[col])
    cleaning_logs.append(f"Capped {outliers} outliers in '{col}' using IQR.")


df.to_csv("cleaned_output.csv", index=False)
cleaning_logs.append("Exported cleaned data to 'cleaned_output.csv'.")

# print all logs
print("\n".join(cleaning_logs))

cleaned_data = pd.read_csv("cleaned_output.csv")
cleaned_data.head(10)

Converted 'Timestamp' and 'Date' to datetime format.
Stripped whitespace from object columns: ['Country', 'Device Category', 'Key Actions', 'Page Path', 'Source', 'Website', 'Time', 'Day']
Standardized casing for 'Country'
Standardized casing for 'Device Category'
Standardized casing for 'Key Actions'
Standardized casing for 'Source'
Standardized casing for 'Website'
Standardized casing for 'Day'
Removed 0 rows with invalid dates.
Capped 0 outliers in 'Avg Session Duration' using IQR.
Capped 0 outliers in 'Bounce Rate' using IQR.
Capped 0 outliers in 'Conversions' using IQR.
Capped 0 outliers in 'New Users' using IQR.
Capped 0 outliers in 'Page Views' using IQR.
Capped 0 outliers in 'Returning Users' using IQR.
Capped 0 outliers in 'Unique Page Views' using IQR.
Capped 0 outliers in 'Average time on home page (min)' using IQR.
Exported cleaned data to 'cleaned_output.csv'.


,Country,Timestamp,Device Category,Key Actions,Page Path,Source,Avg Session Duration,Bounce Rate,Conversions,New Users,Page Views,Returning Users,Unique Page Views,Average time on home page (min),Website,Date,Time,Day
0,india,2019-01-01 05:01:00,mobile,purchase,/home,direct,137.0,35.0,42.0,804.0,1984.0,307.0,243.0,4.41,amazon.com,2019-01-01,05:01:00,tuesday
1,usa,2019-01-01 07:52:00,tablet,sign up,/products,referral,179.0,53.0,15.0,438.0,1938.0,190.0,1194.0,4.89,amazon prime,2019-01-01,07:52:00,tuesday
2,india,2019-02-01 06:55:00,mobile,sign up,/contact,organic search,84.0,49.0,2.0,878.0,1078.0,191.0,450.0,4.35,amazon prime,2019-02-01,06:55:00,wednesday
3,india,2019-03-01 18:09:00,mobile,subscribe,/contact,social media,168.0,35.0,15.0,779.0,372.0,135.0,479.0,5.66,amazon web services (aws),2019-03-01,18:09:00,thursday
4,india,2019-03-01 23:36:00,desktop,contact form,/contact,direct,118.0,42.0,40.0,570.0,1225.0,20.0,147.0,5.19,aws support,2019-03-01,23:36:00,thursday
5,india,2019-04-01 08:46:00,desktop,sign up,/about,direct,160.0,48.0,33.0,208.0,1678.0,167.0,709.0,5.54,amazon music,2019-04-01,08:46:00,friday
6,uk,2019-04-01 09:46:00,mobile,purchase,/blog,organic search,67.0,20.0,4.0,635.0,1291.0,432.0,451.0,0.90,amazon.com,2019-04-01,09:46:00,friday
7,canada,2019-04-01 15:14:00,mobile,sign up,/home,referral,76.0,56.0,19.0,673.0,1271.0,282.0,866.0,2.49,amazon.com,2019-04-01,15:14:00,friday
8,australia,2019-05-01 02:55:00,mobile,download,/about,social media,110.0,42.0,20.0,232.0,1307.0,335.0,1066.0,4.58,prime video,2019-05-01,02:55:00,saturday
9,india,2019-05-01 04:28:00,mobile,purchase,/contact,organic search,136.0,58.0,31.0,83.0,525.0,425.0,1453.0,4.29,amazon.com,2019-05-01,04:28:00,saturday
